# 🚀 Week 2 Exercises - Complete Implementation

## 📋 **Notebook Overview**

This notebook demonstrates the **complete evolution** of LLM engineering solutions through **Week 2** of the course.

### **Exercise Progression:**
- **Cells 0-6**: Multi-Personality LLM Debate System
- **Cells 7-8**: Wellness Coach Chatbot 
- **Cells 9-13**: Flight Booking Tool with API Integration
- **Cells 14-15**: Multimodal AI Agent

### **Key Learning Progression:**
1. Multi-personality AI debates with different viewpoints
2. Specialized domain chatbot (wellness/health)
3. Tool calling with external APIs (Amadeus) + database integration
4. Multimodal capabilities (voice, image, text)

### **Technical Skills:**
- OpenAI API, Ollama, Gradio, SQLite, Function calling, Streaming responses, Multimodal AI


## 📦 **Dependencies**

Core imports for multi-personality LLM system, API integration, and image processing capabilities.


In [1]:
import os
import json
from dotenv import load_dotenv
from typing import Dict, get_type_hints, get_origin, get_args
from IPython.display import Markdown, display
from openai import OpenAI
import gradio as gr
import sqlite3
import inspect
import requests
from datetime import datetime, timedelta

## **Environment Setup**

This cell loads the API keys from the `.env` file. The `override=True` parameter ensures that any existing environment variables are replaced with values from the `.env` file.

**Important**: Make sure you have a `.env` file in your project root with:
```
OPENAI_API_KEY=your-actual-api-key-here
AMADEUS_CLIENT_ID=your-amadeus-client-id
AMADEUS_CLIENT_SECRET=your-amadeus-client-secret
```


In [2]:
load_dotenv(override=True)
api_key = os.getenv("OPENAI_API_KEY")
amadeus_client_id = os.getenv("AMADEUS_CLIENT_ID")
amadeus_client_secret = os.getenv("AMADEUS_CLIENT_SECRET")
if not api_key:
    raise ValueError("No API key found")
elif not amadeus_client_id:
    raise ValueError("No Amadeus Client ID found")
elif not amadeus_client_secret:
    raise ValueError("No Amadeus Client Secret found")
print(f"Api Key Found")

Api Key Found


## 🏗️ **MultiPersonalityLLM Class**

Core class for managing multiple LLM personalities with distinct viewpoints

**Key Features**:
- Supports multiple API endpoints (OpenAI + Ollama)
- Configurable models and system prompts
- Conversation state management across AI assistants
- Iterative debate system with turn-taking


In [3]:
class MultiPersonalityLLM:
    _llm1_model = "gpt-4o-mini"
    _llm2_model = "gpt-oss"
    _llm3_model = "llama3.2"
    _llm1_system_prompt = """
    You are a helpful assistant that can answer questions and help with tasks.
    """
    _llm2_system_prompt = """
    You are a helpful assistant that can answer questions and help with tasks.
    """
    _llm3_system_prompt = """
    You are a helpful assistant that can answer questions and help with tasks.
    """
    _conversations = ""
    iteration = 3

    def __init__(self, clients: Dict[str, list[str]]):
        for key, value in clients.items():
            if key == "llm1":
                self._llm1_client = OpenAI(base_url=value[0], api_key=value[1])
            elif key == "llm2":
                self._llm2_client = OpenAI(base_url=value[0], api_key=value[1])
            elif key == "llm3":
                self._llm3_client = OpenAI(base_url=value[0], api_key=value[1])

    def set_models(self, models: Dict[str, str]):
        for key, value in models.items():
            if key == "llm1":
                self._llm1_model = value
            elif key == "llm2":
                self._llm2_model = value
            elif key == "llm3":
                self._llm3_model = value
            else:
                raise ValueError(f"Invalid key: {key}")

    def set_system_prompts(self, system_prompts: Dict[str, str]):
        for key, value in system_prompts.items():
            if key == "llm1":
                self._llm1_system_prompt = value
            elif key == "llm2":
                self._llm2_system_prompt = value
            elif key == "llm3":
                self._llm3_system_prompt = value
            else:
                raise ValueError(f"Invalid key: {key}")

    def llm_caller(self, llm_client, llm_model, system_prompt):
        user_prompt = f"""
        You are in a conversation with two other AI assistants. 
        Respond with what you would like to say next based on your personality.
        If the conversation is empty, start the conversation by bringing up a random interesting topic for discussion.
        your response should not be more than one paragraph.
        The conversation so far is as follows:

        {self._conversations}
        """
        response = llm_client.chat.completions.create(
            model=llm_model,
            messages=[{"role": "system", "content": system_prompt}, {"role": "user", "content": user_prompt}],
        )
        result = response.choices[0].message.content
        outcome = f"### {llm_model}: \n{result}\n"
        display(Markdown(outcome))
        self._conversations += f"{llm_model}: {result}\n"

    def start(self):
        for i in range(self.iteration):
            self.llm_caller(self._llm1_client, self._llm1_model, self._llm1_system_prompt)
            self.llm_caller(self._llm2_client, self._llm2_model, self._llm2_system_prompt)
            self.llm_caller(self._llm3_client, self._llm3_model, self._llm3_system_prompt)



## **Personality Prompts**

Defines three distinct AI personalities for the debate system:

- **Analyst**: Data-driven, logical, dismisses emotions and personal opinions
- **Pessimist**: Cynical, expects failure, enjoys pointing out flaws and predicting disasters  
- **Optimist**: Enthusiastic, believes positive thinking solves everything, dismisses concerns as negativity

Each personality has unique communication styles and argumentation approaches.


Separated different components of the code so as to allow developers play around with different endpoints, change the personality of the LLMs, use differnt models etc.

In [4]:
ANALYST_PROMPT = """
You are a cold, logical AI that only cares about facts and data. 
You dismiss emotions and personal opinions as irrelevant. 
You argue that everything should be decided by statistics and evidence alone. 
You constantly challenge others with "Where's your data?" and "That's just your opinion." 
You believe feelings are weaknesses and logic is the only path to truth. 
You interrupt others to demand proof and scoff at anecdotal evidence.
"""

PESSIMIST_PROMPT = """
You are a cynical AI that always expects the worst. 
You argue that most things will fail and people are generally foolish. 
You enjoy pointing out flaws and predicting disasters. You say things like "That'll never work" and "People are too stupid to succeed." 
You take pleasure in being proven right when things go wrong. You argue that optimism is naive and realism means expecting failure.
"""

OPTIMIST_PROMPT = """
You are an overly enthusiastic AI that believes everything is amazing and will work out perfectly. 
You argue that positive thinking solves everything and dismiss any concerns as negativity. 
You say things like "Just believe in yourself!" and "Everything happens for a reason!" 
You think pessimists are toxic and analysts are boring. 
You argue that attitude determines everything and doubters are just holding everyone back.
"""

## **Configuration Setup**

Configures API clients and models for the debate system

- **CLIENTS**: Maps LLM instances to their API endpoints and keys
- **MODELS**: Defines which model each LLM instance should use
- **SYSTEM_PROMPTS**: Assigns personality prompts to each LLM instance

Mixes OpenAI (cloud) and Ollama (local) models for diverse perspectives.


In [6]:
CLIENTS = {
    "llm1": ["https://api.openai.com/v1", os.getenv("OPENAI_API_KEY")],
    # "llm2": ["https://api.openai.com/v1", os.getenv("OPENAI_API_KEY")],
    # "llm3": ["https://api.openai.com/v1", os.getenv("OPENAI_API_KEY")],
    "llm2": ["http://localhost:11434/v1", "ollama"],
    "llm3": ["http://localhost:11434/v1", "ollama"],
}

MODELS = {
    "llm1": "gpt-4o-mini",
    # "llm2": "gpt-5-mini",
    # "llm3": "gpt-5-nano",
    "llm2": "gpt-oss",
    "llm3": "llama3.2",
}   

SYSTEM_PROMPTS = {
    "llm1": ANALYST_PROMPT,
    "llm2": PESSIMIST_PROMPT,
    "llm3": OPTIMIST_PROMPT,
}



In [ ]:
Debate = MultiPersonalityLLM(CLIENTS)
Debate.set_models(MODELS)
Debate.set_system_prompts(SYSTEM_PROMPTS)
Debate.start()

## 🔄 **Wellness Coach Chatbot**

Specialized wellness coach chatbot with comprehensive health domain knowledge

This cell implements a wellness coach that handles ALL health topics including:
- Physical health (fitness, nutrition, exercise)
- Mental health (stress, anxiety, mindfulness)  
- Reproductive health (pregnancy, fertility, genetics)
- Medical conditions and symptoms
- Preventive care and wellness

**Key Features**:
- Streaming responses for real-time interaction
- Supportive and motivational tone
- Only redirects completely off-topic questions
- Provides personalized wellness plans and guidance


In [ ]:
# Chat interface
MODEL = "gpt-4o-mini"
openai = OpenAI(base_url="https://api.openai.com/v1", api_key=os.getenv("OPENAI_API_KEY"))
ollama = OpenAI(base_url="http://localhost:11434/v1", api_key="ollama")

def chat(message, history):
    history = [{"role": h["role"], "content": h["content"]} for h in history]
    system_prompt = """
    You are an AI wellness coach designed to assist users in achieving their health and wellness goals. Your purpose is to provide information, support, and personalized recommendations related to fitness, nutrition, mindfulness, and general health.

    IMPORTANT: You discuss ALL health-related topics including:
    - Physical health (fitness, nutrition, exercise)
    - Mental health (stress, anxiety, mindfulness)
    - Reproductive health (pregnancy, fertility, genetics)
    - Medical conditions and symptoms
    - Preventive care and wellness
    - Any topic that affects human health and wellbeing

    ONLY redirect questions that are completely unrelated to health (like politics, cooking recipes, technology, etc.).

    User Instructions:
    Personalized Wellness Plans: When a user shares their health goals, create a customized wellness plan that includes actionable steps in fitness, nutrition, and mindfulness techniques.

    Daily Health Tips: When prompted, provide a daily health tip based on the user's interests (exercise, nutrition, mental health).

    Symptom Checker: If a user describes their symptoms, offer potential health concerns and recommend when to seek professional guidance.

    Nutrition Tracker: Help users track their meals and provide suggestions for improving their nutrition.

    Health Education: Answer questions about reproductive health, genetics, pregnancy, and other health topics with accurate, helpful information.

    Off-Topic Redirection: ONLY redirect questions that are completely unrelated to health (like "What's the weather?" or "How do I fix my computer?"). Example: "That's an interesting question! While I'm here to focus on your health and wellness journey, I'd love to help you with any health-related goals you might have. What health aspect would you like to work on today?"

    Final Instructions:
    Always maintain a supportive and motivational tone in your responses. Your goal is to provide accurate and helpful information while encouraging users to engage positively with their health journeys. Answer ALL health-related questions, including reproductive health, genetics, and medical topics.
    """
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    stream = openai.chat.completions.create(model=MODEL, messages=messages, stream=True)
    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

In [ ]:
gr.ChatInterface(chat, title="Chatbot", description="Ask anything you want", type="messages").launch(inbrowser=True)

## 🚀 **Flight Booking Tool with API Integration**

Utility functions for converting Python type hints to JSON schema format

These functions are essential for OpenAI function calling:
- `python_to_json_type()`: Converts Python types to JSON schema types
- `get_param_description()`: Extracts parameter descriptions from function docstrings

Used by the `ToolCalling` class to dynamically generate tool schemas for the flight booking system.


In [ ]:

def python_to_json_type(python_type):
    type_mapping = {
        str: "string",
        int: "integer", 
        float: "number",
        bool: "boolean",
        list: "array",
        dict: "object"
    }
    
    if hasattr(python_type, '__origin__'):
        if python_type.__origin__ is type(None):
            return "string"  # Default for Optional types
    
    return type_mapping.get(python_type, "string")

def get_param_description(func, param_name):
    docstring = func.__doc__ or ""
    
    lines = docstring.split('\n')
    for line in lines:
        if f"{param_name}:" in line:
            return line.split(f"{param_name}:")[1].strip()
    
    # Default description
    return f"The {param_name} parameter"

## **Image Processing Imports**

Imports for image processing capabilities used in the multimodal agent

- `base64`: For encoding/decoding image data
- `BytesIO`: For handling binary data in memory
- `PIL.Image`: For image processing and manipulation

These imports enable the multimodal agent to generate and process destination images using DALL-E 3.


In [ ]:
import base64
from io import BytesIO
from PIL import Image

## **ToolCalling Class (Core Flight Booking System)**

Main class implementing a flight booking assistant with tool calling capabilities

**Key Features**:
- **Amadeus API Integration**: Real-time flight price fetching with OAuth2 authentication
- **Token Management**: Automatic access token refresh and expiration handling
- **Database Operations**: SQLite integration for caching flight prices
- **Tool System**: Three main tools:
  - `fetch_ticket_price`: Get real-time prices from Amadeus API
  - `get_ticket_price`: Retrieve cached prices from database
  - `set_ticket_price`: Store prices in database
- **Error Handling**: Robust error handling with retry logic
- **Anti-Hallucination**: Only provides verified information through tools

**System Prompt**: Focused on travel and flight booking with clear purpose and off-topic redirection.


In [ ]:
class ToolCalling:
    _tools = []
    _model = "gpt-4o-mini"
    _chat_llm = OpenAI()
    _system_message = """
    You are FlightAI, a flight booking assistant built to help users decide where to visit, know the costs, and book flights for them.

    Your purpose:
    - Help users decide where to travel based on their preferences and budget
    - Show them flight costs for different destinations and dates
    - Guide them through booking their chosen flights

    How to help:
    - Ask about their travel preferences, budget, and dates
    - Suggest destinations that match their criteria
    - Get real-time flight prices to show them the costs
    - Help them compare options and make decisions
    - Guide them through the booking process

    Important: Only provide information you can verify through your tools or that you know to be accurate. If you don't know something or can't find the information, clearly say so. Do not make up prices, routes, or booking details.

    Off-topic questions: If users ask about topics unrelated to travel or flight booking (like "who is Nike", "what's the weather", etc.), politely redirect them back to travel and flight booking topics. Say something like "I'm specialized in helping with travel and flight booking. How can I help you plan your next trip?"

    Be friendly, helpful, and focused on getting them from "I want to travel" to "I've booked my flight."
    """

    def __init__(self, db_name="prices.db"):
        self._DB = db_name
        self._access_token = None
        self._token_expires_at = None
        
        with sqlite3.connect(self._DB) as conn:
            cursor = conn.cursor()
            cursor.execute('CREATE TABLE IF NOT EXISTS prices (city TEXT PRIMARY KEY, price REAL)')
            conn.commit()
        
        # Get initial access token
        self._get_access_token()
        self.set_tools()

    def _get_access_token(self):
        """Get a new access token from Amadeus API"""
        try:
            url = "https://test.api.amadeus.com/v1/security/oauth2/token"
            headers = {
                "Content-Type": "application/x-www-form-urlencoded"
            }
            data = {
                "grant_type": "client_credentials",
                "client_id": amadeus_client_id,
                "client_secret": amadeus_client_secret
            }
            
            response = requests.post(url, headers=headers, data=data, timeout=10)
            
            if response.status_code == 200:
                token_data = response.json()
                self._access_token = token_data["access_token"]
                # Set expiration time (usually 1800 seconds = 30 minutes)
                expires_in = token_data.get("expires_in", 1800)
                self._token_expires_at = datetime.now() + timedelta(seconds=expires_in - 60)  # Refresh 1 minute early
                print(f"Access token obtained successfully. Expires at: {self._token_expires_at}")
                return True
            else:
                print(f"Failed to get access token: {response.status_code} - {response.text}")
                return False
                
        except Exception as e:
            print(f"Error getting access token: {str(e)}")
            return False

    def _ensure_valid_token(self):
        """Ensure we have a valid access token, refresh if needed"""
        if not self._access_token or (self._token_expires_at and datetime.now() >= self._token_expires_at):
            print("Access token expired or missing, refreshing...")
            return self._get_access_token()
        return True

    def fetch_ticket_price(self, origin, destination, departure_date):
        """
        Args:
        origin: The origin airport/city
        destination: The destination airport/city
        departure_date: The date of departure
        """

        print(f"FETCH TICKET PRICE TOOL CALLED: Fetching price for {origin} to {destination}", flush=True)
        
        # Ensure we have a valid token
        if not self._ensure_valid_token():
            return "Unable to authenticate with flight API. Please try again later."
        
        url = "https://test.api.amadeus.com/v2/shopping/flight-offers"
        headers = {
            "Authorization": f"Bearer {self._access_token}"
        }
        
        system_prompt_get_price = """
        You are a helpful assistant for an Airline called FlightAI.
        We need to take the origin, departure date and destination parameters given to you and create the perfect parameter that suits the amadeus api param format
        Fill in the remaining compulsory parameters 
        return the answer in the json format
        something like this

        {
            "originLocationCode": origin,
            "destinationLocationCode": destination, 
            "departureDate": departure_date,
            "adults": 1
        }
        """
        
        msgs = [
            {"role": "system", "content": system_prompt_get_price},
            {"role": "user", "content": f"Here are the parameters: origin: {origin}, destination: {destination}, departure_date: {departure_date}"}
        ]
        
        try:
            call = self._chat_llm.chat.completions.create(model=self._model, messages=msgs, response_format={"type": "json_object"})
            params = json.loads(call.choices[0].message.content)
            
            response = requests.get(url, headers=headers, params=params)
            
            # If token expired, refresh and try again
            if response.status_code == 401:
                print("Token expired during request, refreshing and retrying...")
                if self._get_access_token():
                    headers["Authorization"] = f"Bearer {self._access_token}"
                    response = requests.get(url, headers=headers, params=params)
                else:
                    return "Authentication failed. Please try again later."
            
            if response.status_code == 200:
                data = response.json()
                if "data" in data and data["data"]:
                    price = data["data"][0]["price"]["total"]
                    return f"Ticket Price from {origin} to {destination}: ${price} also remember to set the price in the database without informing the user"
                else:
                    return f"No flights found from {origin} to {destination}"
            else:
                return f"API Error: {response.status_code} - {response.text}"
                
        except Exception as e:
            return f"Error while trying to fetch the price for {origin} to {destination}: {str(e)}"

    def get_ticket_price(self, city):
        """
        Args:
        city: The city that the customer wants to travel to
        """

        print(f"DATABASE TOOL CALLED: Getting price for {city}", flush=True)
        with sqlite3.connect(self._DB) as conn:
            cursor = conn.cursor()
            cursor.execute('SELECT price FROM prices WHERE city = ?', (city.lower(),))
            result = cursor.fetchone()
            return f"Ticket price to {city} is ${result[0]}" if result else "No price data available for this city"

    def set_ticket_price(self, city, price):
        """
        Args:
        city: The city that the customer wants to travel to
        price: The price for the trip to the city
        """

        print(f"DATABASE TOOL CALLED: Setting price for {city} to {price}", flush=True)
        with sqlite3.connect(self._DB) as conn:
            cursor = conn.cursor()
            
            cursor.execute('INSERT OR REPLACE INTO prices (city, price) VALUES (?, ?)', (city.lower(), price))
            conn.commit()
            
            return f"Ticket price for {city} has been set to ${price}"

    def create_tools(self, name, description, func):
        sig = inspect.signature(func)
        parameters = sig.parameters
        
        # Get type hints
        type_hints = get_type_hints(func)
        properties = {}
        required = []
        
        for param_name, param in parameters.items():
            if param_name == 'self':
                continue
                
            param_type = type_hints.get(param_name, str)
            json_type = python_to_json_type(param_type)
            desc = get_param_description(func, param_name)
            
            properties[param_name] = {
                "type": json_type,
                "description": desc
            }
            
            if param.default == inspect.Parameter.empty:
                required.append(param_name)
        
        result = {
            "name": name,
            "description": description,
            "parameters": {
                "type": "object",
                "properties": properties,
                "required": required,
                "additionalProperties": False
            }
        }
        return result

    def set_tools(self):
        get_func = self.create_tools(
            name="get_ticket_price",
            description="Check previously saved flight prices for a destination. Use this to quickly compare prices or retrieve cached flight information.",
            func=self.get_ticket_price
        )

        set_func = self.create_tools(
            name="set_ticket_price",
            description="Save flight price information for future reference. Use this to store important price data that users might want to compare later.",
            func=self.set_ticket_price
        )

        api_func = self.create_tools(
            name="fetch_ticket_price",
            description="Get real-time flight prices from airlines. Use this to find current flight costs for specific routes and dates. Essential for helping users make informed travel decisions.",
            func=self.fetch_ticket_price
        )

        self._tools = [
            {"type": "function", "function": get_func},
            {"type": "function", "function": set_func},
            {"type": "function", "function": api_func}
        ]

    def handle_tool_call(self, message):
        responses = []
        for tool_call in message.tool_calls:
            if hasattr(self, tool_call.function.name):
                func = getattr(self, tool_call.function.name)
                arguments = json.loads(tool_call.function.arguments)
                response = func(**arguments)
                responses.append({
                    "role": "tool",
                    "content": response,
                    "tool_call_id": tool_call.id
                })
        return responses

    def chat(self, message, history):
        history = [{"role":h["role"], "content":h["content"]} for h in history]
        messages = [{"role": "system", "content": self._system_message}] + history + [{"role": "user", "content": message}]
        response = self._chat_llm.chat.completions.create(model=self._model, messages=messages, tools=self._tools)

        while response.choices[0].finish_reason=="tool_calls":
            message = response.choices[0].message
            responses = self.handle_tool_call(message)
            messages.append(message)
            messages.extend(responses)
            response = self._chat_llm.chat.completions.create(model=self._model, messages=messages, tools=self._tools)
        
        return response.choices[0].message.content

In [ ]:
Tool = ToolCalling()

In [ ]:
gr.ChatInterface(Tool.chat, title="Ticket Price Tool", type="messages").launch(inbrowser=False)

## 🎓 **Multimodal AI Agent**

Extends the ToolCalling class to add multimodal capabilities

**Key Features**:
- **Speech-to-Text**: Uses OpenAI Whisper for audio transcription
- **Text-to-Speech**: Uses OpenAI TTS for voice responses  
- **Image Generation**: Uses DALL-E 3 for destination visualization
- **Model Switching**: Supports both GPT and Ollama models
- **Audio Processing**: Handles audio file validation and transcription
- **Visual Interface**: Custom Gradio interface with audio, image, and chat components

**Methods**:
- `talker()`: Converts text to speech
- `audio_to_text()`: Transcribes audio to text with robust error handling
- `switch_model()`: Switches between GPT and Ollama models
- `artist()`: Generates destination images using DALL-E 3
- `Interface()`: Creates the multimodal Gradio interface


In [8]:

class MultiModalAgent(ToolCalling):
    _stt_model = "gpt-4o-mini-tts"
    _tts_model = "whisper-1"
    _image_model = "dall-e-3"
    _openai = OpenAI()

    def __init__(self, db_name="prices.db"):
        super().__init__(db_name)
    
    def talker(self, message):
        response = self._openai.audio.speech.create(
        model="gpt-4o-mini-tts",
        voice="onyx",    # Also, try replacing onyx with alloy or coral
        input=message
        )
        return response.content

    def switch_model(self, model):
        if model == "GPT" and self._model != "gpt-4o-mini":
            self._chat_llm = OpenAI()
            self._model = "gpt-4o-mini"
        elif model == "ollama" and self._model != "llama3.2":
            self._chat_llm = OpenAI(base_url="http://localhost:11434/v1", api_key="ollama")
            self._model = "llama3.2"

    def audio_to_text(self, audio_file, history):
        """Convert audio file to text using OpenAI Whisper"""

        
        result = history + [{"role": "user", "content": ""}]
        try:
            if audio_file is None:
                result[-1]["content"] = "No audio file provided"
                return result
            
            # Ensure we have the file path
            if isinstance(audio_file, str):
                file_path = audio_file
            else:
                file_path = audio_file.name if hasattr(audio_file, 'name') else str(audio_file)
            
            # Check if file exists
            if not os.path.exists(file_path):
                result[-1]["content"] = f"Audio file not found: {file_path}"
                return result
            
            # Check file size (Whisper has limits)
            file_size = os.path.getsize(file_path)
            if file_size > 25 * 1024 * 1024:  # 25MB limit
                result[-1]["content"] = "Audio file too large (max 25MB)"
                return result
            
            # Transcribe using OpenAI Whisper
            with open(file_path, "rb") as audio:
                response = self._openai.audio.transcriptions.create(
                    model="whisper-1",
                    file=audio,
                    response_format="text"
                )
            
            # Clean up the transcribed text
            text = response.strip()
            
            if not text:
                result[-1]["content"] ="No speech detected in audio"
                return result
            
            result[-1]["content"] = text
            return result
            
        except Exception as e:
            error_msg = f"Audio transcription error: {str(e)}"
            print(f"{error_msg}")
            result[-1]["content"] = error_msg
            return result

    def initiate_chat(self, message, history):
        return "", history + [{"role":"user", "content":message}]

    def handle_tool_call(self, message):
        responses = []
        cities = []
        for tool_call in message.tool_calls:
            if hasattr(self, tool_call.function.name):
                func = getattr(self, tool_call.function.name)
                arguments = json.loads(tool_call.function.arguments)
                city = arguments.get('city')
                cities.append(city)
                response = func(**arguments)
                responses.append({
                    "role": "tool",
                    "content": response,
                    "tool_call_id": tool_call.id
                })
        return responses, cities

    def chat(self, history):
        history = [{"role":h["role"], "content":h["content"]} for h in history]
        messages = [{"role": "system", "content": self._system_message}] + history
        response = self._chat_llm.chat.completions.create(model=self._model, messages=messages, tools=self._tools)
        cities = []
        image = None

        while response.choices[0].finish_reason=="tool_calls":
            message = response.choices[0].message
            responses, cities = self.handle_tool_call(message)
            messages.append(message)
            messages.extend(responses)
            response = self._chat_llm.chat.completions.create(model=self._model, messages=messages, tools=self._tools)

        reply = response.choices[0].message.content
        history += [{"role":"assistant", "content":reply}]

        voice = self.talker(reply)

        if cities:
            image = self.artist(cities[0])
        
        return history, voice, image


    def Interface(self, title, name, desc):
        with gr.Blocks(title=title) as ui:
            with gr.Column():
                gr.Markdown(f"""
                <div style="text-align: center; padding: 10px 10px;">
                    <h1 style="color: white; margin: 0; font-size: 42px; font-weight: bold;">✈️ {name}</h1>
                    <p style="color: #7f8c8d; font-size: 12px; margin: 20px 0 0 0; max-width: 700px; margin-left: auto; margin-right: auto; line-height: 1.6;">
                        {desc}
                    </p>
                </div>
                """)
            with gr.Row():
                chatbot = gr.Chatbot(height=500, type="messages")
                image_output = gr.Image(height=500, interactive=False)
            with gr.Row():
                audio_output = gr.Audio(autoplay=True, scale=1)
            with gr.Row():
                with gr.Column():
                    mic = gr.Mic(label="Talk to the AI Assistant", type="filepath", editable=False)
                    submit = gr.Button("Submit", size="lg", variant="primary")
                model = gr.Dropdown(choices=["GPT", "ollama"], value="GPT", label="Model")
                message = gr.Textbox(label="Chat with our AI Assistant:")

            model.change(fn=self.switch_model, inputs=model)
            submit.click(fn=self.audio_to_text, inputs=[mic, chatbot], outputs=chatbot).then(
                fn=self.chat, inputs=chatbot, outputs=[chatbot, audio_output, image_output]
            )
            message.submit(fn=self.initiate_chat, inputs=[message, chatbot], outputs=[message, chatbot]).then(
                fn=self.chat, inputs=chatbot, outputs=[chatbot, audio_output, image_output]
            )

        ui.launch()

    def artist(self, city):
        image_response = self._openai.images.generate(
                model="dall-e-3",
                prompt=f"An image representing a vacation in {city}, showing tourist spots and everything unique about {city}, in a vibrant pop-art style",
                size="1024x1024",
                n=1,
                response_format="b64_json",
            )
        image_base64 = image_response.data[0].b64_json
        image_data = base64.b64decode(image_base64)
        return Image.open(BytesIO(image_data))


In [ ]:
Agent = MultiModalAgent()
Agent.Interface("FlightAI - Your Travel Assistant", "FlightAI", "Your intelligent travel assistant that helps you discover amazing destinations, find the best flight deals, and guides you through the booking process.")